# Project Strongest - 回収率シミュレーション

このNotebookでは、訓練済みの `EnsembleModel` を使用して馬券回収率のシミュレーションを行い、ベースライン戦略（1番人気に賭け続ける）と比較します。

In [ ]:
import sys
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import softmax

# srcをパスに追加
if 'src' not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), '../src'))

from model.ensemble import EnsembleModel

%matplotlib inline
# 日本語フォント設定（環境に合わせて適宜変更してください）
sns.set(font='MSGothic') 
sns.set_style('whitegrid')

## 1. データとモデルのロード

In [ ]:
# 前処理済みデータ（オッズを含む生データ）のロード
data_path = os.path.join(os.getcwd(), '../data/processed/preprocessed_data.parquet')
if not os.path.exists(data_path):
    raise FileNotFoundError(f"データファイルが見つかりません: {data_path}")

df = pd.read_parquet(data_path)
# 2024年のデータをテストセットとして使用 (evaluate.py のロジックに準拠)
test_df = df[df['year'] == 2024].copy()
print(f"テストデータ (2024年): {len(test_df)} 行")

In [ ]:
# モデルのロード
model_path = os.path.join(os.getcwd(), '../models/ensemble_model.pkl')
if not os.path.exists(model_path):
    raise FileNotFoundError(f"モデルファイルが見つかりません: {model_path}")

model = EnsembleModel()
model.load_model(model_path)
print("モデルのロードに成功しました")

In [ ]:
# データセットのメタデータをロード（使用する特徴量を取得するため）
dataset_path = os.path.join(os.getcwd(), '../data/processed/lgbm_datasets.pkl')
with open(dataset_path, 'rb') as f:
    datasets = pickle.load(f)

feature_cols = datasets['train']['X'].columns.tolist()
print(f"特徴量数: {len(feature_cols)}")

## 2. 予測の実行

In [ ]:
X_test = test_df[feature_cols]
scores = model.predict(X_test)
test_df['score'] = scores

# 確率と期待値の計算
test_df['prob'] = test_df.groupby('race_id')['score'].transform(lambda x: softmax(x))
test_df['expected_value'] = test_df['prob'] * test_df['odds'].fillna(0)

test_df[['race_id', 'horse_name', 'odds', 'score', 'prob', 'expected_value']].head()

## 3. シミュレーションロジック
各レースで最もスコア（Score）が高い馬に100円を賭けるシミュレーションを行います。

In [ ]:
def simulate_betting(df, strategy_col='score', bet_amount=100):
    results = []
    
    # レースごとにグループ化
    grouped = df.groupby('race_id')
    
    for race_id, group in grouped:
        if group[strategy_col].isnull().all():
            continue
            
        # 馬を選択
        best_horse_idx = group[strategy_col].idxmax()
        best_horse = group.loc[best_horse_idx]
        
        # 結果判定
        is_hit = (best_horse['rank'] == 1)
        return_amt = best_horse['odds'] * bet_amount if is_hit else 0
        
        results.append({
            'race_id': race_id,
            'date': best_horse['date'],
            'bet': bet_amount,
            'return': return_amt,
            'hit': 1 if is_hit else 0,
            'odds': best_horse['odds'],
            'horse_name': best_horse['horse_name']
        })
        
    return pd.DataFrame(results)

# 戦略1: AIモデル (最大スコア)
sim_ai = simulate_betting(test_df, strategy_col='score')
sim_ai['strategy'] = 'AI Model'

# 戦略2: ベースライン (1番人気)
# 注: popularity は 1 が一番人気（値が小さいほど人気がある）。
# idxmax ロジックを使うため、マイナスを掛けて反転させる
test_df['neg_popularity'] = -test_df['popularity']
sim_base = simulate_betting(test_df, strategy_col='neg_popularity')
sim_base['strategy'] = 'Baseline (1st Popular)'

## 4. 可視化

### 4.1 全体サマリー

In [ ]:
def print_summary(sim_df, name):
    total_bet = sim_df['bet'].sum()
    total_return = sim_df['return'].sum()
    roi = total_return / total_bet * 100
    accuracy = sim_df['hit'].mean() * 100
    print(f"--- {name} ---")
    print(f"総投資額: {total_bet} 円")
    print(f"総回収額: {total_return:.0f} 円")
    print(f"回収率 (ROI): {roi:.2f}%")
    print(f"的中率: {accuracy:.2f}%")
    print("")

print_summary(sim_ai, "AIモデル")
print_summary(sim_base, "ベースライン (1番人気)")

### 4.2 累積収益の推移
収支が時間の経過とともにどのように推移するかを可視化します。

In [ ]:
# プロット用にデータを結合
sim_ai['cumsum_bet'] = sim_ai['bet'].cumsum()
sim_ai['cumsum_return'] = sim_ai['return'].cumsum()
sim_ai['balance'] = sim_ai['cumsum_return'] - sim_ai['cumsum_bet']

sim_base['cumsum_bet'] = sim_base['bet'].cumsum()
sim_base['cumsum_return'] = sim_base['return'].cumsum()
sim_base['balance'] = sim_base['cumsum_return'] - sim_base['cumsum_bet']

plt.figure(figsize=(14, 7))
plt.plot(pd.to_datetime(sim_ai['date']), sim_ai['balance'], label='AI Model', linewidth=2)
plt.plot(pd.to_datetime(sim_base['date']), sim_base['balance'], label='Baseline (1st Popular)', linestyle='--', alpha=0.7)
plt.axhline(0, color='black', linewidth=0.5)
plt.title('累積収支の推移 (2024年)')
plt.xlabel('日付')
plt.ylabel('収支 (円)')
plt.legend()
plt.show()

### 4.3 月別回収率の比較

In [ ]:
sim_ai['month'] = pd.to_datetime(sim_ai['date']).dt.to_period('M')
monthly_roi = sim_ai.groupby('month').apply(lambda x: x['return'].sum() / x['bet'].sum() * 100)

plt.figure(figsize=(12, 6))
monthly_roi.plot(kind='bar', color='skyblue')
plt.axhline(100, color='red', linestyle='--', label='損益分岐点 (100%)')
plt.title('月別回収率 (AIモデル)')
plt.ylabel('回収率 (%)')
plt.legend()
plt.show()